In [1]:
import gradio as gr
import requests
import json

# The API key will be automatically provided by the Canvas environment if left as an empty string.
# DO NOT hardcode your API key here in a real application for security reasons.
# If you were running this code outside of Canvas, you would replace "" with your actual API key.
API_KEY = ""

def my_google_llm(cuisine_type, meal_type):
    """
    This function takes a cuisine type and a meal type, then uses the Google Gemini API
    to suggest 5 food options.

    Args:
        cuisine_type (str): The type of cuisine (e.g., 'Italian', 'Indian').
        meal_type (str): The type of meal (e.g., 'lunch', 'dinner', 'breakfast').

    Returns:
        str: 5 food options or an error message if something goes wrong.
    """
    try:
        # Construct the prompt for the LLM based on user input
        # Now asking for 5 specific food options based on cuisine and meal type
        user_message_content = (
            f"For a {meal_type} meal, focusing on {cuisine_type} cuisine, "
            f"please suggest 5 distinct and delicious food options. "
            f"List them clearly, perhaps with a short description for each."
        )

        # Define the message structure for the Gemini API
        payload = {
            "contents": [
                {
                    "role": "user",
                    "parts": [
                        # UPDATED SYSTEM PROMPT:
                        # Instructs the AI to act as a food expert and provide specific options,
                        # explicitly stating to give 5 distinct food items.
                        {"text": "You are an AI assistant specialized in culinary recommendations. When asked for food options based on cuisine and meal type, you will generate 5 distinct and delicious food options. Always provide specific dish names and brief descriptions. Do not suggest using external search engines or disclaim lack of real-time data."},
                        {"text": user_message_content}
                    ]
                }
            ]
        }

        # Define the API URL for the gemini-1.5-flash model
        api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key="

        # Make the POST request to the Gemini API
        headers = {"Content-Type": "application/json"}
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        
        # Raise an HTTPError for bad responses (4xx or 5xx)
        response.raise_for_status() 

        # Parse the JSON response from the API
        result = response.json()

        # Extract the generated text content from the response
        if (result.get("candidates") and len(result["candidates"]) > 0 and
            result["candidates"][0].get("content") and
            result["candidates"][0]["content"].get("parts") and
            len(result["candidates"][0]["content"]["parts"]) > 0):
            return result["candidates"][0]["content"]["parts"][0]["text"]
        else:
            # Handle cases where the response structure is unexpected or content is missing
            return "Could not retrieve valid food options from the AI model. The response structure was unexpected."

    except requests.exceptions.RequestException as e:
        # Catch network-related errors or bad HTTP responses
        return f"Error connecting to the Gemini API: {e}. Please check your internet connection or API key."
    except json.JSONDecodeError:
        # Catch errors if the response is not valid JSON
        return "Error decoding the API response. The response might not be in a valid JSON format."
    except Exception as e:
        # Catch any other unexpected errors
        return f"An unexpected error occurred: {e}"

# Create the Gradio interface for the food recommendation assistant
pr = gr.Interface(
    fn=my_google_llm,
    inputs=[
        gr.Textbox(label="Cuisine Type (e.g., 'Italian', 'Indian', 'Mexican')", lines=1),
        gr.Textbox(label="Meal Type (e.g., 'breakfast', 'lunch', 'dinner', 'snack')", lines=1)
    ],
    outputs="text",
    title="🍽️ Food Option Suggester 🧑‍🍳",
    description="Tell me the cuisine and meal type, and I'll suggest 5 delicious food options!"
)

# Launch the Gradio application
pr.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
